# Project 1: computing all kinematic informations of a real manipulator

In this project (and the next ones), we will build all the necessary various functionalities for realistic robot manipulators. This first project aims to build the core fonctions (basic homogeneous transforms, twists, forward kinematics and Jacobians) that will be a foundation for all subsequent algorithms.

## Instructions
* Answer all questions in the notebook
* You will need to submit on Brightspace: 
    1. the code you wrote to answer the questions in a Jupyter Notebook. The code should be runnable as is.
    2. a 2-3 pages report in pdf format (pdf only) detailing the methodology you followed to answer the questions as well as answers to the questions that require a written answer. You may add the plots in the report (does not count for the page limit) or in the Jupyter notebook.


## The robot

We will use a model of the [Kuka iiwa 14 robot](https://www.kuka.com/en-us/products/robotics-systems/industrial-robots/lbr-iiwa). This robot has 7 revolute joints and its kinematics is described in the picture below:

![](https://raw.githubusercontent.com/righetti/ROB6003/143afa17d7eb1af79c3f6ce034973a1774da5d42/Project1/kuka_kinematics.png "Kuka iiwa 14 Kinematic Model")

# Setup

Run the cell below only once when resetting the runtime in Colab - this will not do anything when running on a local Jupyter Notebook.

In [1]:
## check if we are in Google Colab
try:
    import google.colab
    RUNNING_IN_COLAB = True
    print('detected Colab - setting up environment')
    # then we need to install the conda environment
    try:
        import condacolab
        condacolab.check()
    except:
        !pip install -q condacolab
        import condacolab
        condacolab.install()
except:
    RUNNING_IN_COLAB = False

In [2]:
# after installing condalab, the runtime restarts
# -> need to check for colab env once more here
try:
    import google.colab
    RUNNING_IN_COLAB = True
except Exception as e:
    RUNNING_IN_COLAB = False

if RUNNING_IN_COLAB:
    try:
        # Check if packages are installed or not. If not, install them.
        import pinocchio
    except:
        # Install pinocchio, meschat-python 
        !conda install pinocchio meshcat-python
       
    # get the class repo - first check if it exists
    import os, sys
    if not os.path.isdir('/content/ROB6003/Project1'):
      print('cloning LAB repository')
      os.chdir('/content')
      !git clone https://github.com/righetti/ROB6003.git
      print('cloning done')
    else:
      print('lab repos was found, skipping cloning')
    print('done configuring for Colab')
    sys.path.append('/content/ROB6003/Project1/')
    os.chdir('/content/ROB6003/Project1/')
    print('done adding system path and changing directory.')


# Starting the visualization environment

The following code will start a visualization environment (click on the printed address to see the robot)

You need to run this only ONCE. Each time you run this cell you will get a new display environment (so you need to close the previous one!)

This should work out of the box on Google Colab and you local Jupyter Notebook (make sure you have installed the right libraries in your local computer if you do not use Colab).

In [3]:
import numpy as np
import robot_visualizer
import time
from scipy.linalg import expm
from scipy.linalg import logm
import matplotlib.pyplot as plt

robot_visualizer.start_robot_visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You should see the Kuka iiwa robot now when going to this page: http://127.0.0.1:7000/static/


# Displaying an arbitrary configuration

You can use the following function to display arbitrary configurations of the robot

In [4]:
# here we display an arbitrary configuration of the robot
q = np.random.sample([7])
print(f'we show the configuration for the angles {q}')
robot_visualizer.display_robot(q)

we show the configuration for the angles [0.62980371 0.26192933 0.85209457 0.93205156 0.85534565 0.87379974
 0.76184097]


## Question 1: basics
In this first set of questions, we aim to write the basic functions to do kinematics
* Write a function ``vec_to_skew(w)`` that transforms a 3D vector (numpy array) into a skew symmetric matrix
* Write a function ``twist_to_skew(V)`` that transforms a 6D twist into a 4x4 matrix (use ``vec_to_skew``)
* Write a function ``exp_twist_bracket(V)`` that returns the exponential of a (bracketed) twist $\mathrm{e}^{[\mathcal{V}]}$ where the input to the function is a 6D twist
* Write a function ``inverseT(T)`` that returns the inverse of a homogeneous transform T
* Write a function ``getAdjoint(T)`` that returns the adjoint of a homogeneous transform T

In [5]:
 """1.1 
 function_name : vec_to_skew 
 arguments     : 3D vector -w
 return_value  : skew symmetric matrix
 description   : Given a 3D vector (numpy array),vec_to_skew() converts it
                into a skew symmetric matrix
"""
def vec_to_skew(w):
    return np.array([[0, -w.item(2), w.item(1)],
                         [w.item(2), 0, -w.item(0)],
                         [-w.item(1), w.item(0), 0]])
"""1.2 
 function_name : twist_to_skew 
 arguments     : Twist V
 return value  : skew matrix
 description   : Given a 6D twist V, twist_to_skew() converts it
                 into a 4x4 skew matrix

"""
def twist_to_skew(V):
    return np.row_stack((np.column_stack((vec_to_skew(np.array(V[0:3,0])),
                                        np.array(V[3:6,0]))),np.zeros((1,4))))
       
"""1.3
 function_name : exp_twist_bracket 
 arguments     : Twist V (6D)
 return value  : exponential of a twist
 description   : Given a 6D twist V, exp_twist_bracket() converts it
                 into a the exponential of a (bracketed) twist 

"""  
def exp_twist_bracket(V):
    return  expm(twist_to_skew(V))

"""1.4
 function_name : inverseT
 arguments     : a homogeneous transform T
 return value  : inverse of homogeneous transform
 description   : Given a homogeneous transform, inverseT() converts it
                 into its inverse
                 
""" 
def inverseT(T):
    return np.linalg.inv(T)

"""1.5
 function_name : getAdjoint
 arguments     : a homogeneous transform T
 return value  : adjoint of homogeneous transform
 description   : Given a homogeneous transform, getAdjoint() converts it
                 into its adjoint matrix
                 
"""
def getAdjoint(T):
    R  =  np.array(T[0:3,0:3])
    p  = vec_to_skew(np.array(T[0:3,3]))
    return np.array(np.row_stack((np.column_stack((R,np.zeros((3,3)))),
                               np.column_stack((p@R,R)))))


In [6]:


"""
Class_name : joints
attributes :
              S       : Screw matrix of the joint  (Unit Twist)
              T       : Homogeneous transform (Pose of joint in s frame)
              eStheta : exponential of S*theta (theta - joint variable)
              s       : screw of the joint with applied transformation
              Js      : space jacobian of the joint
methods    :
              method_name :_init_
              arguments   :
                          self - joint object
                          S    - Screw of joint
                          T    - Homogeneous transform of joint
              description : _init_()initializes Screw matrix 
                            and Homogeneous transform for the given joint
"""

class joints:
    S = np.array([[0],[0],[0],[0],[0],[0]])
    T = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
    eStheta = np.array([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])
    s = np.array([[0],[0],[0],[0],[0],[0]])
    Js = np.array([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0]])
    def __init__(self,S, T):
        self.S = S
        self.T = T
        self.s = getAdjoint(self.T)@self.S 

#Initializing joints

#distance from s frame
d1  = 0.1575
d2  = d1 + 0.2025
d3  = d2 + 0.2045
d4  = d3 + 0.2155
d5  = d4 + 0.1845
d6z = d5 + 0.2155 
d6y = 0.0607
d7  = d6z+0.0810
dh  = d7 + 0.04

# Initializing screw and homogeneous matrices of joints in an array
Joint_Arr = []
Joint_Arr.append(joints(np.array([[0],[0],[1],[0],[0],[0]]),
              np.array([[1,0,0,0],[0,1,0,0],[0,0,1,d1],[0,0,0,1]])))
Joint_Arr.append(joints(np.array([[0],[1],[0],[0],[0],[0]]),
              np.array([[1,0,0,0],[0,1,0,0],[0,0,1,d2],[0,0,0,1]])))
Joint_Arr.append(joints(np.array([[0],[0],[1],[0],[0],[0]]),
              np.array([[1,0,0,0],[0,1,0,0],[0,0,1,d3],[0,0,0,1]])))
Joint_Arr.append(joints(np.array([[0],[-1],[0],[0],[0],[0]]),
              np.array([[1,0,0,0],[0,1,0,0],[0,0,1,d4],[0,0,0,1]])))
Joint_Arr.append(joints(np.array([[0],[0],[1],[0],[0],[0]]),
              np.array([[1,0,0,0],[0,1,0,0],[0,0,1,d5],[0,0,0,1]])))
Joint_Arr.append(joints(np.array([[0],[1],[0],[0],[0],[0]]),
              np.array([[1,0,0,0],[0,1,0,d6y],[0,0,1,d6z],[0,0,0,1]])))
Joint_Arr.append(joints(np.array([[0],[0],[1],[0],[0],[0]]),
              np.array([[1,0,0,0],[0,1,0,0],[0,0,1,d7],[0,0,0,1]])))

#Pose of end effector in spatial frame
M = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,dh],[0,0,0,1]])


## Question 2: forward kinematics
* Write a function ``forward_kinematics(theta)`` that gets as an input an array of joint angles and computes the pose of the end-effector.

In order to test this function, you are given the following forward kinematics results (up to $10^{-4}$ precision),

$T_{SH}(0,\ 0,\ 0,\ 0,\ 0,\ 0,\ 0) = \begin{bmatrix}1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 1.301 \end{bmatrix}$,

$T_{SH}(0.2,\ -0.2,\ 0.5,\ -0.4,\ 1.2,\ -0.8,\ 0.4) = \begin{bmatrix}
-0.4951 & -0.814 &  0.3037 & -0.0003 \\
0.6286 & -0.5769 & -0.5215 &  0.0056\\
0.5997 & -0.0673 &  0.7974 &  1.2563\\
0.  &    0.  &    0.  &    1.\end{bmatrix}$

$T_{SH}(-1.2,\ 0.7,\ 2.8,\ 0.7,\ 1.2,\ 0.2,\ 0.3) = \begin{bmatrix}
-0.9669 & -0.254 &  -0.0234 &  0.1535\\
0.0976 & -0.2835 & -0.954 &  -0.7557\\
0.2357 & -0.9247 &  0.2989 &  0.795\\
 0.  &    0.  &    0.  &    1.\end{bmatrix}$

In [7]:
"""2
 function_name : forward_kinematics
 arguments     : an array of joint variables
 return value  : forward kinematics of the kuka robot
 description   : Given an array of joint variables of the robot,
                 forward_kinematics() computes the forward kinematics
                 of the robot it
                 
"""
def forward_kinematics(theta):
    Tsh = np.eye(4)
    for x in range(len(Joint_Arr)):
        Joint_Arr[x].eStheta = exp_twist_bracket(Joint_Arr[x].s*theta[x])
        Tsh = Tsh @ Joint_Arr[x].eStheta
    return Tsh@M


In [8]:
#Testing Forward Kinematics
t1  = np.array([0,0,0,0,0,0,0])
t2  =  np.array([0.2,-0.2,0.5,-0.4, 1.2, -0.8,0.4])
t3  = np.array([-1.2, 0.7, 2.8,0.7,1.2,0.2,0.3])

FK1 = forward_kinematics(t1)
FK2 = forward_kinematics(t2)
FK3 = forward_kinematics(t3)

print("*****************************************************************")
print("Configuration 1")
print("")
print("theta")
print(t1)
print("")
print("Forward kinematics")
print(FK1 )
print("")
print("*****************************************************************")
print("")
print("*****************************************************************")
print("Configuration 2")
print("")
print("theta")
print(t2)
print("")
print("Forward kinematics")
print(FK2 )
print("")
print("*****************************************************************")
print("")
print("*****************************************************************")
print("Configuration 3")
print("")
print("theta")
print(t3)
print("")
print("Forward kinematics")
print(FK3)
print("")
print("*****************************************************************")

*****************************************************************
Configuration 1

theta
[0 0 0 0 0 0 0]

Forward kinematics
[[1.    0.    0.    0.   ]
 [0.    1.    0.    0.   ]
 [0.    0.    1.    1.301]
 [0.    0.    0.    1.   ]]

*****************************************************************

*****************************************************************
Configuration 2

theta
[ 0.2 -0.2  0.5 -0.4  1.2 -0.8  0.4]

Forward kinematics
[[-4.95108167e-01 -8.14041264e-01  3.03652306e-01 -3.04658056e-04]
 [ 6.28646572e-01 -5.76891624e-01 -5.21535753e-01  5.58217963e-03]
 [ 5.99726096e-01 -6.73266294e-02  7.97368005e-01  1.25634777e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

*****************************************************************

*****************************************************************
Configuration 3

theta
[-1.2  0.7  2.8  0.7  1.2  0.2  0.3]

Forward kinematics
[[-0.96690733 -0.2540491  -0.02343634  0.1534601 ]
 [ 0.0976152  -0.2

## Question 3: jacobians
* Write a function ``get_space_jacobian(theta)`` that computes the space jacobian given an array of joint angles

In order to test this function, you are given the following space Jacobian results (up to $10^{-3}$ precision),
$J^S(0,\ 0,\ 0,\ 0,\ 0,\ 0,\ 0) = \begin{bmatrix}
   0.000 &   0.000 &   0.000 &   0.000 &   0.000 &   0.000 &   0.000\\
   0.000 &   1.000 &   0.000 & -1.000 &   0.000 &   1.000 &   0.000\\
   1.000 &   0.000 &   1.000 &   0.000 &   1.000 &   0.000 &   1.000\\
   0.000 & -0.360 &   0.000 &   0.780 &   0.000 & -1.180 &   0.000\\
   0.000 &   0.000 &   0.000 &   0.000 &   0.000 &   0.000 &   0.000\\
   0.000 &   0.000 &   0.000 &   0.000 &   0.000 &   0.000 &   0.000
\end{bmatrix}
$,

$J^S(0.2,\ -0.2,\ 0.5,\ -0.4,\ 1.2,\ -0.8,\ 0.4) = \begin{bmatrix}
   0.000 & -0.199 & -0.195 &   0.635 &   0.112 & -0.943 &   0.304\\
   0.000 &   0.980 & -0.039 & -0.767 &   0.213 & -0.287 & -0.522\\
   1.000 &   0.000 &   0.980 &   0.095 &   0.971 &   0.172 &   0.797\\
   0.000 & -0.353 &   0.014 &   0.590 & -0.181 &   0.344 &   0.660\\
   0.000 & -0.072 & -0.070 &   0.498 &   0.166 & -1.087 &   0.382\\
   0.000 &   0.000 &   0.000 &   0.073 & -0.016 &   0.075 & -0.002
\end{bmatrix}$

$J^S(-1.2,\ 0.7,\ 2.8,\ 0.7,\ 1.2,\ 0.2,\ 0.3) = \begin{bmatrix}
   0.000 &   0.932 &   0.233 &   0.971 &   0.146 & -0.528 & -0.023\\
   0.000 &   0.362 & -0.600 &   0.103 & -0.970 & -0.242 & -0.954\\
   1.000 &   0.000 &   0.765 & -0.216 &   0.194 & -0.814 &   0.299\\
   0.000 & -0.130 &   0.216 & -0.015 &   0.612 &   0.705 &   0.533\\
   0.000 &   0.336 &   0.084 &   0.683 &   0.080 & -0.274 & -0.065\\
   0.000 &   0.000 &   0.000 &   0.255 & -0.058 & -0.376 & -0.164
\end{bmatrix}$

In [9]:

"""3 
 function_name : get_space_jacobian
 arguments     : an array of joint variables
 return value  : Space Jacobian
 description   : Given an array of joint variables of the robot,
                 get_space_jacobian() computes the space jacobian
                 
"""
def get_space_jacobian(theta):
    J1 = np.array([])
    for x in range(len(Joint_Arr)):
        if (x == 0):
            M1 = np.eye(4)
        else:
            M1 = M1@exp_twist_bracket(Joint_Arr[x-1].s*theta[x-1])
        Joint_Arr[x].Js =  getAdjoint(M1)@Joint_Arr[x].s
        if (x == 0):
            J1 = Joint_Arr[x].Js 
        else:
            J1 = np.column_stack((J1,Joint_Arr[x].Js ))
    return J1



In [10]:
#Testing Jacobian
J1  = get_space_jacobian(t1)
J2  = get_space_jacobian(t2)
J3  = get_space_jacobian(t3)

print("*****************************************************************")
print("Configuration 1")
print("")
print("theta")
print(t1)
print("")
print("Jacobian")
print(J1 )
print("")
print("*****************************************************************")
print("")
print("*****************************************************************")
print("Configuration 2")
print("")
print("theta")
print(t2)
print("")
print("Jacobian")
print(J2 )
print("")
print("*****************************************************************")
print("")
print("*****************************************************************")
print("Configuration 3")
print("")
print("theta")
print(t3)
print("")
print("Jacobian")
print(J3 )
print("")
print("*****************************************************************")

*****************************************************************
Configuration 1

theta
[0 0 0 0 0 0 0]

Jacobian
[[ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.    0.   -1.    0.    1.    0.  ]
 [ 1.    0.    1.    0.    1.    0.    1.  ]
 [ 0.   -0.36  0.    0.78  0.   -1.18  0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]]

*****************************************************************

*****************************************************************
Configuration 2

theta
[ 0.2 -0.2  0.5 -0.4  1.2 -0.8  0.4]

Jacobian
[[ 0.00000000e+00 -1.98669331e-01 -1.94709171e-01  6.34851591e-01
   1.11828162e-01 -9.42585858e-01  3.03652306e-01]
 [ 0.00000000e+00  9.80066578e-01 -3.94695030e-02 -7.66740789e-01
   2.13162993e-01 -2.86545866e-01 -5.21535753e-01]
 [ 1.00000000e+00  0.00000000e+00  9.80066578e-01  9.52471509e-02
   9.70595694e-01  1.71532410e-01  7.97368005e-01]
 [ 0.00000000e+00 -3.52823968e-01  1.42090211e-02  5.900597

### Hint: Q2 and Q3
Depending on which method you use to compute the quantities of Q2 and Q3, you will need to define a series of fixed homogeneous transforms, screws, etc. You may want to store these values in various variables that you can reuse (i.e. define the fixed kinematic parameters once and for all).

You may also want to store some intermediate results to later compute the Jacobians.

Feel free to design the data structure that you prefer and to also pass additional parameters or return multiple variables with these functions if it simplifies your design. You can also put these functions in a class if you wish. Any solution is ok, as long as you can compute the requested quantities.

Make sure to explain your design in the report.

## Question 4: displaying hand trajectories 
You are given a file ``joint_trajectory.npy`` which contains a sequence of 100 joint configurations (cf. below) corresponding to a motion of the robot over time.
* Compute the position of the hand (i.e. the origin of the frame H) in the spatial frame for all 100 joint configuration
* Plot x-y position of the hand for all the configurations (i.e. a 2D plot with x as the abscissa and y as the ordinate of the graph). What does the hand draw?
* Do the same analysis and plots for the x-z and y-z pairs.

### Hint
You may use (matplotlib)[https://matplotlib.org/] to draw plots

In [11]:
#we open the file and put all the data in the variable joint_trajectory 
#this gives us a 7 x 200 array (each column in one set of joint configurations)
with open('joint_trajectory.npy', 'rb') as f:
    joint_trajectory = np.load(f)
    

n_samples = joint_trajectory.shape[1]
FK = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
FKcfg = []


for i in range(n_samples):
    FK = forward_kinematics(joint_trajectory[0:7,i])
    if(i == 0):
        TE  = np.array(FK[0:3,3])
    else:
        TE = np.column_stack((TE,np.array(FK[0:3,3])))
    FKcfg.append(FK)
    # we display the trajectory
    robot_visualizer.display_robot(joint_trajectory[:,i])
    time.sleep(0.05) # we wait between two displays so we can see each configuration

#Extracting x,y,z from Kinematics computed    
xe = np.array(TE[0])
ye = np.array(TE[1])
ze = np.array(TE[2])  


# Plotting Hand positions

In [ ]:

plt.plot(xe,ye,linewidth=4)
plt.xlabel('x',fontsize=20)
plt.ylabel('y',fontsize=20)
plt.title('Position of the Hand x-y position',fontsize=30)


In [ ]:
plt.plot(xe,ze,linewidth=4)
plt.xlabel('x',fontsize=20)
plt.ylabel('z',fontsize=20)

plt.title('Position of the Hand - x-z position',fontsize=30)



In [ ]:
plt.figure(figsize=[7,5])
plt.plot(ye,ze,linewidth=4)
plt.xlabel('y',fontsize=20)
plt.ylabel('z',fontsize=20)
plt.title('Position of the Hand - y-z position',fontsize=25)

In [ ]:
# now we plot the joint trajectories for each joint (each cross correspond to one data point)
plt.figure(figsize=[10,15])
for i in range(7):
    plt.subplot(7,1,i+1)
    plt.plot(joint_trajectory[i,:], 'x', linewidth=4)
    plt.ylabel(f'joint {i+1}', fontsize=30)

## Question 5: computing velocities
The file ``joint_velocities.npy`` contains the velocities of each joint corresponding to the sequence joint configurations seen in the previous question. 
* Use the Jacobian to compute the linear velocity of the endeffector in: 1) the spatial frame. 2) the end-effector frame and 3) in a frame with same origin as the end-effector frame but oriented like the spatial frame
* Plot these velocities in each frame (one plot per dimension x,y,z)
* Compare these plots and relate them to the plot of the positions (Question 4), is there a frame that seems most intuitive to you? Why?

In [ ]:
"""
 function_name : extractlinvel
 arguments     : Twist V
 return value  : linear velocity
 description   : Given a Twist,extractlinvel()
                 extracts the linear velocity
                 
"""
def extractlinvel(V):
    return np.array([[V.item(3)],[V.item(4)],[V.item(5)]])
"""
 function_name : computeVnfromJs
 arguments     : space jacobian Js, homogeneous transform T 
                 and joint velocities dtheta
 return value  : Twist 
 description   : Given space jacobian,homogeneous transform T 
                 and joint velocities dtheta, computeVnfromJs()
                 computes the Twist
                 
"""
def computeVnfromJs(Js,T,dtheta):
    Tinv = inverseT(T)
    AT   = getAdjoint(Tinv)
    Jn   = AT@Js
    return np.array(Jn@dtheta)


# we open the file and put all the data in the variable joint_trajectory 
# this gives us a 7 x 200 array (each column in one set of joint configurations)
with open('joint_velocity.npy', 'rb') as f:
    joint_velocities = np.load(f)

Range = joint_velocities.shape[1]
Js = np.array([[0],[0],[0],[0],[0],[0],[0]])
Jh = np.array([[0],[0],[0],[0],[0],[0],[0]])
last = np.array([0,0,0,1])
FKcfg = np.array(FKcfg)

for i in range(Range):
    Js = get_space_jacobian(joint_trajectory[0:7,i])
    #Constructing T for the new frame   
    first = np.column_stack((np.eye(3),TE[:,i]))
    T_hyb = np.row_stack((first,last))
    #Computing Twists using Jacobians
    Vs   = np.array(Js@joint_velocities[0:7,i])
    Vh   = computeVnfromJs(Js,FKcfg[i,:,:],joint_velocities[0:7,i])
    Vhyb = computeVnfromJs(Js,T_hyb,joint_velocities[0:7,i])
    #Extracting linear velocity
    if (i== 0):    
        v_in_s   = extractlinvel(Vs)
        v_in_h   = extractlinvel(Vh)
        v_in_hyb = extractlinvel(Vhyb)
    else:
        v_in_s   = np.column_stack((v_in_s, extractlinvel(Vs)))
        v_in_h   = np.column_stack((v_in_h, extractlinvel(Vh)))
        v_in_hyb = np.column_stack((v_in_hyb,extractlinvel(Vhyb)))
    
vsx = v_in_s[0]
vsy = v_in_s[1]
vsz = v_in_s[2]

vhx = v_in_h[0]
vhy = v_in_h[1]
vhz = v_in_h[2]

vhsx = v_in_hyb[0]
vhsy = v_in_hyb[1]
vhsz = v_in_hyb[2]



In [ ]:

plt.figure(figsize=[9,12])        

plt.subplot(3,1,1)
plt.title('velocity in x direction',fontsize=25)
plt.plot(vsx,linewidth=2)
plt.subplot(3,1,2)
plt.title('velocity in y direction',fontsize=25)
plt.plot(vsy,linewidth=2)
plt.subplot(3,1,3)
plt.plot(vsz,linewidth=2)
plt.title('velocity in z direction',fontsize=25)
plt.suptitle('Linear velocity in s frame',fontsize=30,y=1)
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25,wspace=0.35)

In [ ]:
plt.figure(figsize=[10,12]) 
plt.subplot(3,1,1)
plt.plot(vhx,linewidth=2)
plt.title('velocity in x direction',fontsize=25)
plt.subplot(3,1,2)
plt.plot(vhy,linewidth=2)
plt.title('velocity in y direction',fontsize=25)
plt.subplot(3,1,3)
plt.plot(vhz,linewidth=2)
plt.title('velocity in z direction',fontsize=25)
plt.suptitle('Linear velocity in h frame',fontsize=30,y=1)
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25,wspace=0.35)


In [ ]:
plt.figure(figsize=[9,11]) 
plt.subplot(3,1,1)
plt.plot(vhsx,linewidth=2)
plt.title('velocity in x direction',fontsize=25)
plt.subplot(3,1,2)
plt.plot(vhsy,linewidth=2)
plt.title('velocity in y direction',fontsize=25)
plt.subplot(3,1,3)
plt.plot(vhsz,linewidth=2)
plt.title('velocity in z direction',fontsize=25)
plt.suptitle('Linear velocity in the frame oriented like s and positioned at h',fontsize=30,y=1)
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25,wspace=0.35)

In [ ]:
#Plotting velocity and position 
plt.plot(vsx,label = 'velocity x component')
plt.plot(vsy,label = 'velocity y component')
plt.plot(vsz,label = 'velocity z component')
plt.plot(xe,label = ' x position')
plt.plot(ye,label = ' y position')
plt.plot(ze,label = ' z position')
plt.title('S frame')
plt.legend()

In [ ]:
#Plotting velocity and position 
plt.plot(vhsx,label = 'velocity x component')
plt.plot(vhsy,label = 'velocity y component')
plt.plot(vhsz,label = 'velocity z component')
plt.plot(xe,label = ' x position')
plt.plot(ye,label = ' y position')
plt.plot(ze,label = ' z position')
plt.title('Frame oriented in S and positioned in H')
plt.legend()

In [ ]:
# we open the file and put all the data in the variable joint_trajectory 
# this gives us a 7 x 200 array (each column in one set of joint configurations)
with open('joint_velocity.npy', 'rb') as f:
    joint_velocities = np.load(f)
    
# now we plot the joint velocities for each joint (each cross correspond to one data point)
plt.figure(figsize=[10,15])
for i in range(7):
    plt.subplot(7,1,i+1)
    plt.plot(joint_velocities[i,:], 'x', linewidth=4)
    plt.ylabel(f'joint {i+1}', fontsize=30)